# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097005


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:00,  2.17s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.30s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:40,  1.63s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:26,  1.17s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:00,  7.14it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  8.58it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  9.11it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00, 10.70it/s]

OregonHiker                           0.000089
Linda_J._Berkel                       0.000827
jnarayanbvg                           0.005909
awright5                              0.000125
not-logged-in-c6917d9b19f41e6c2995    0.005995
twolf35                               0.048146
rmz4160                               0.000212
bentleyghioda                         0.000112
Nauter                                0.000207
not-logged-in-8e0ce538cfce67a074d4    0.000206
Meckert7597                           0.000154
not-logged-in-5f30b2bb9b7f9d49da41    0.024889
not-logged-in-2cac8dc00d6857aef75c    0.001325
not-logged-in-5f30b2bb9b7f9d49da41    0.014119
AdamMorse                             0.000283
not-logged-in-c016e89f1952c96c30e6    0.025434
not-logged-in-4abb07bb689832abb57b    0.013793
not-logged-in-986357d841d09bf87b26    0.000125
pangeli5                              0.000431
not-logged-in-a959612abf1827865481    0.000396
Lavadude                              0.042904
helios1235   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())